# 🌍 Global CO₂ Emissions – Comprehensive ML Analysis

**Dataset:** Our World in Data – Global CO₂ and Greenhouse Gas Emissions (1750–2021)

| Stat | Value |
|---|---|
| Rows | ~50,600 country-year records |
| Columns | 79 features |
| Countries | 278 (countries + aggregates) |
| Period | 1750 – 2021 |

**ML Tasks:** CO₂ Regression · Emission-Level Classification · Country Clustering · Growth-Rate Forecasting

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns
import warnings, os, base64
from io import BytesIO

from sklearn.model_selection import (train_test_split, cross_val_score,
    StratifiedKFold, GridSearchCV, RandomizedSearchCV, learning_curve)
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report, roc_curve, auc,
    mean_squared_error, mean_absolute_error, r2_score, silhouette_score)
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier,
    RandomForestRegressor, GradientBoostingRegressor,
    VotingClassifier, StackingClassifier, AdaBoostClassifier)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA

warnings.filterwarnings("ignore")
sns.set_style("whitegrid")
plt.rcParams["figure.dpi"] = 100
plt.rcParams["font.size"] = 11

SEED = 42
np.random.seed(SEED)
os.makedirs("outputs/plots", exist_ok=True)

report_images = {}
results = {}          # classification results
regression_results = {}
growth_results = {}   # growth-rate regression

def save_plot(fig, name):
    """Save figure to PNG and store base64 for the HTML report."""
    fig.savefig(f"outputs/plots/{name}.png", dpi=150, bbox_inches="tight")
    buf = BytesIO()
    fig.savefig(buf, format="png", dpi=150, bbox_inches="tight")
    buf.seek(0)
    report_images[name] = base64.b64encode(buf.read()).decode("utf-8")
    plt.close(fig)
    print(f"  \u2713 {name}.png")

print("All imports successful")
print(f"  scikit-learn {__import__('sklearn').__version__}")
print(f"  pandas {pd.__version__}, numpy {np.__version__}")

## 1. Data Loading & Exploration

In [ ]:
DATA = "CO2+Emissions/visualizing_global_co2_data.csv"
raw = pd.read_csv(DATA)
print(f"Raw dataset: {raw.shape[0]:,} rows x {raw.shape[1]} columns")
print(f"Year range:  {raw['year'].min()} – {raw['year'].max()}")
print(f"Countries:   {raw['country'].nunique()}")

# Show missingness
miss = raw.isnull().mean().sort_values(ascending=False)
print(f"\nTop-15 missing-value columns (%):")
for c, v in miss.head(15).items():
    print(f"  {c:45s} {v*100:5.1f}%")

raw.head()

In [ ]:
# Separate actual countries from aggregates (continents, income groups, etc.)
# Aggregates have no iso_code or iso_code in special codes
aggregates = raw[raw["iso_code"].isna() | raw["iso_code"].str.startswith("OWID")]
countries  = raw[~raw.index.isin(aggregates.index)].copy()
print(f"Country rows: {len(countries):,}  |  Aggregate rows: {len(aggregates):,}")
print(f"Unique countries: {countries['country'].nunique()}")

# Focus on modern era with enough data
df = countries[countries["year"] >= 1950].copy()
print(f"\nFiltered to 1950+: {len(df):,} rows, {df['country'].nunique()} countries")

# Key columns for analysis
core_cols = ["country","year","iso_code","population","gdp",
             "co2","co2_per_capita","coal_co2","oil_co2","gas_co2",
             "cement_co2","flaring_co2","other_industry_co2",
             "land_use_change_co2","co2_including_luc",
             "co2_growth_prct","co2_per_gdp",
             "primary_energy_consumption","energy_per_capita",
             "methane","nitrous_oxide","total_ghg",
             "temperature_change_from_co2","temperature_change_from_ghg",
             "share_global_co2","cumulative_co2"]

df = df[[c for c in core_cols if c in df.columns]].copy()
print(f"Selected {df.shape[1]} core columns")
print(f"Missing values: {df.isnull().sum().sum():,} total ({df.isnull().mean().mean()*100:.1f}% avg)")

## 2. Data Cleaning & Feature Engineering

Filter to 1950+, fill/drop missing values, and create derived features.

In [ ]:
# Drop rows with no CO2 value (our primary target)
df = df.dropna(subset=["co2"]).copy()
print(f"After dropping NaN co2: {len(df):,} rows")

# Fill GDP / population with 0 (will be excluded in models needing them)
df["gdp"] = df["gdp"].fillna(0)
df["population"] = df["population"].fillna(0)

# Fill source-specific CO2 columns with 0 (no data = no emissions from that source)
source_cols = ["coal_co2","oil_co2","gas_co2","cement_co2","flaring_co2",
               "other_industry_co2","land_use_change_co2"]
for c in source_cols:
    if c in df.columns:
        df[c] = df[c].fillna(0)

# ---- Feature Engineering ----
# Fuel mix percentages
total_fuel = df["coal_co2"] + df["oil_co2"] + df["gas_co2"] + 0.001  # avoid /0
df["coal_share"] = df["coal_co2"] / total_fuel
df["oil_share"]  = df["oil_co2"]  / total_fuel
df["gas_share"]  = df["gas_co2"]  / total_fuel

# GDP per capita (where gdp and population > 0)
df["gdp_per_capita"] = np.where(df["population"] > 0, df["gdp"] / df["population"], 0)

# Decade
df["decade"] = (df["year"] // 10) * 10

# CO2 intensity (co2 per unit of energy, where available)
df["co2_per_gdp"] = df["co2_per_gdp"].fillna(0)

# Log-transform skewed targets
df["log_co2"] = np.log1p(df["co2"])
df["log_population"] = np.log1p(df["population"])

# Emission level categories (for classification)
co2_pc = df["co2_per_capita"].fillna(0)
df["emission_level"] = pd.cut(co2_pc,
    bins=[-np.inf, 1, 5, 15, np.inf],
    labels=["Low","Medium","High","Very High"])

print(f"Final dataset: {df.shape}")
print(f"\nEmission Level distribution:")
print(df["emission_level"].value_counts().sort_index())

df.to_csv("outputs/co2_cleaned.csv", index=False)
print("\nCleaned dataset saved.")

## 3. Exploratory Data Analysis

In [ ]:
# ---- Fig 1: Global CO2 over time ----
global_ts = df.groupby("year")["co2"].sum().reset_index()
fig, ax = plt.subplots(figsize=(14, 5))
ax.fill_between(global_ts["year"], global_ts["co2"], alpha=0.3, color="steelblue")
ax.plot(global_ts["year"], global_ts["co2"], color="steelblue", linewidth=2)
ax.set_title("Global CO\u2082 Emissions Over Time (1950\u20132021)", fontweight="bold")
ax.set_xlabel("Year"); ax.set_ylabel("CO\u2082 (million tonnes)")
fig.tight_layout()
save_plot(fig, "global_co2_timeline")

# ---- Fig 2: Top 10 emitters (latest year) ----
latest = df[df["year"] == df["year"].max()]
top10 = latest.nlargest(10, "co2")
fig, ax = plt.subplots(figsize=(12, 5))
colors = sns.color_palette("viridis", 10)
bars = ax.barh(top10["country"][::-1], top10["co2"].values[::-1], color=colors)
ax.set_title(f"Top 10 CO\u2082 Emitters ({df['year'].max()})", fontweight="bold")
ax.set_xlabel("CO\u2082 (million tonnes)")
for b, v in zip(bars, top10["co2"].values[::-1]):
    ax.text(b.get_width()+50, b.get_y()+b.get_height()/2, f"{v:,.0f}", va="center", fontsize=9)
fig.tight_layout()
save_plot(fig, "top10_emitters")

# ---- Fig 3: CO2 by source over time ----
fig, ax = plt.subplots(figsize=(14, 6))
by_year = df.groupby("year")[["coal_co2","oil_co2","gas_co2","cement_co2","flaring_co2"]].sum()
ax.stackplot(by_year.index, by_year["coal_co2"], by_year["oil_co2"],
             by_year["gas_co2"], by_year["cement_co2"], by_year["flaring_co2"],
             labels=["Coal","Oil","Gas","Cement","Flaring"],
             colors=sns.color_palette("Set2", 5), alpha=0.85)
ax.set_title("Global CO\u2082 by Fuel Source", fontweight="bold")
ax.set_xlabel("Year"); ax.set_ylabel("CO\u2082 (million tonnes)")
ax.legend(loc="upper left")
fig.tight_layout()
save_plot(fig, "co2_by_source")

# ---- Fig 4: CO2 per capita distribution ----
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
valid_pc = latest[latest["co2_per_capita"].notna() & (latest["co2_per_capita"] > 0)]
axes[0].hist(valid_pc["co2_per_capita"], bins=40, color="steelblue", edgecolor="white")
axes[0].set_title("CO\u2082 per Capita Distribution (Latest Year)", fontweight="bold")
axes[0].set_xlabel("Tonnes per person"); axes[0].set_ylabel("Countries")
# Emission level pie
el = df["emission_level"].value_counts().sort_index()
axes[1].pie(el.values, labels=el.index, autopct="%1.1f%%",
            colors=["#2ecc71","#f39c12","#e74c3c","#8e44ad"], startangle=140)
axes[1].set_title("Emission Level Distribution", fontweight="bold")
fig.tight_layout()
save_plot(fig, "co2_per_capita_dist")

# ---- Fig 5: Correlation heatmap ----
num_cols = ["co2","co2_per_capita","coal_co2","oil_co2","gas_co2",
            "cement_co2","population","gdp","gdp_per_capita",
            "coal_share","oil_share","gas_share","log_co2"]
corr = df[num_cols].corr()
fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap="coolwarm",
            center=0, ax=ax, square=True, linewidths=0.5, annot_kws={"size": 8})
ax.set_title("Feature Correlation Heatmap", fontweight="bold")
fig.tight_layout()
save_plot(fig, "correlation_heatmap")

# ---- Fig 6: Top 5 countries timeline ----
fig, ax = plt.subplots(figsize=(14, 6))
top5_names = top10["country"].head(5).tolist()
for name in top5_names:
    cdf = df[df["country"] == name]
    ax.plot(cdf["year"], cdf["co2"], linewidth=2, label=name)
ax.set_title("CO\u2082 Emissions \u2013 Top 5 Countries", fontweight="bold")
ax.set_xlabel("Year"); ax.set_ylabel("CO\u2082 (million tonnes)")
ax.legend(); fig.tight_layout()
save_plot(fig, "top5_timeline")

print("All EDA plots saved!")

## 4. CO₂ Emissions Prediction (Regression)

Predict total CO₂ emissions from economic, demographic, and energy features.

In [ ]:
# Build regression dataset – rows with GDP, population, and energy data
reg_df = df[(df["gdp"] > 0) & (df["population"] > 0)].copy()
print(f"Regression dataset: {len(reg_df):,} rows")

reg_features = ["population","gdp","gdp_per_capita","year",
                "coal_co2","oil_co2","gas_co2","cement_co2","flaring_co2"]
X_r = reg_df[reg_features].values
y_r = reg_df["co2"].values

X_r_tr, X_r_te, y_r_tr, y_r_te = train_test_split(
    X_r, y_r, test_size=0.2, random_state=SEED)
sc_r = StandardScaler()
X_r_tr_s = sc_r.fit_transform(X_r_tr)
X_r_te_s = sc_r.transform(X_r_te)
print(f"Train: {len(y_r_tr):,}  |  Test: {len(y_r_te):,}")

def evaluate_reg(name, model, Xtr, Xte, ytr, yte, store=regression_results):
    model.fit(Xtr, ytr)
    yp = model.predict(Xte)
    r2  = r2_score(yte, yp)
    mae = mean_absolute_error(yte, yp)
    rmse = np.sqrt(mean_squared_error(yte, yp))
    store[name] = {"r2": r2, "mae": mae, "rmse": rmse, "y_pred": yp, "model": model}
    print(f"  {name:30s}  R\u00b2={r2:.4f}  MAE={mae:.2f}  RMSE={rmse:.2f}")
    return model

print("\nCO\u2082 Regression Results:")
print("=" * 80)
evaluate_reg("Linear Regression", LinearRegression(), X_r_tr_s, X_r_te_s, y_r_tr, y_r_te)
evaluate_reg("Ridge Regression", Ridge(alpha=1.0), X_r_tr_s, X_r_te_s, y_r_tr, y_r_te)
evaluate_reg("Lasso Regression", Lasso(alpha=1.0, max_iter=5000), X_r_tr_s, X_r_te_s, y_r_tr, y_r_te)
evaluate_reg("Decision Tree", DecisionTreeRegressor(max_depth=15, random_state=SEED),
             X_r_tr, X_r_te, y_r_tr, y_r_te)
rf_reg = evaluate_reg("Random Forest",
    RandomForestRegressor(n_estimators=200, max_depth=20, random_state=SEED, n_jobs=-1),
    X_r_tr, X_r_te, y_r_tr, y_r_te)
gb_reg = evaluate_reg("Gradient Boosting",
    GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=5, random_state=SEED),
    X_r_tr, X_r_te, y_r_tr, y_r_te)

# Actual vs Predicted
best_reg = max(regression_results, key=lambda k: regression_results[k]["r2"])
yp_best = regression_results[best_reg]["y_pred"]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].scatter(y_r_te, yp_best, alpha=0.2, s=8, color="steelblue")
lim = max(y_r_te.max(), yp_best.max())
axes[0].plot([0, lim], [0, lim], "r--", linewidth=1)
axes[0].set_xlabel("Actual CO\u2082"); axes[0].set_ylabel("Predicted CO\u2082")
axes[0].set_title(f"Actual vs Predicted \u2013 {best_reg}", fontweight="bold")

reg_comp = pd.DataFrame({n: {k: v for k, v in d.items() if k not in ("y_pred","model")}
                          for n, d in regression_results.items()}).T.sort_values("r2")
reg_comp["r2"].plot(kind="barh", ax=axes[1], color="steelblue")
axes[1].set_title("Regression R\u00b2 Comparison", fontweight="bold")
axes[1].set_xlabel("R\u00b2"); axes[1].set_xlim(0, 1.05)
fig.tight_layout()
save_plot(fig, "regression_results")

# Feature importance for RF regressor
rf_imp = pd.Series(rf_reg.feature_importances_, index=reg_features)
fig, ax = plt.subplots(figsize=(10, 5))
rf_imp.sort_values().plot(kind="barh", ax=ax, color="steelblue")
ax.set_title("Random Forest Regressor \u2013 Feature Importance", fontweight="bold")
ax.set_xlabel("Importance")
fig.tight_layout()
save_plot(fig, "regression_feature_importance")

print(f"\nBest: {best_reg} (R\u00b2 = {regression_results[best_reg]['r2']:.4f})")

## 5. Emission-Level Classification

Classify country-year records into **Low / Medium / High / Very High** emission levels
based on per-capita CO₂.

In [ ]:
# Prepare classification dataset
clf_df = df.dropna(subset=["emission_level"]).copy()
clf_df = clf_df[(clf_df["gdp"] > 0) & (clf_df["population"] > 0)]
print(f"Classification dataset: {len(clf_df):,} rows")
print(clf_df["emission_level"].value_counts().sort_index())

le_el = LabelEncoder()
y_c = le_el.fit_transform(clf_df["emission_level"])
class_names = le_el.classes_
print(f"Classes: {list(class_names)}")

clf_features = ["population","gdp","gdp_per_capita","year",
                "coal_share","oil_share","gas_share",
                "co2","coal_co2","oil_co2","gas_co2",
                "log_population","log_co2"]
X_c = clf_df[clf_features].values

X_c_tr, X_c_te, y_c_tr, y_c_te = train_test_split(
    X_c, y_c, test_size=0.2, random_state=SEED, stratify=y_c)
sc_c = StandardScaler()
X_c_tr_s = sc_c.fit_transform(X_c_tr)
X_c_te_s = sc_c.transform(X_c_te)
print(f"\nTrain: {len(y_c_tr):,}  |  Test: {len(y_c_te):,}")

def evaluate_clf(name, model, Xtr, Xte, ytr, yte, store=results):
    model.fit(Xtr, ytr)
    yp = model.predict(Xte)
    acc  = accuracy_score(yte, yp)
    prec = precision_score(yte, yp, average="weighted", zero_division=0)
    rec  = recall_score(yte, yp, average="weighted", zero_division=0)
    f1   = f1_score(yte, yp, average="weighted", zero_division=0)
    store[name] = {"accuracy": acc, "precision": prec, "recall": rec,
                   "f1_score": f1, "y_pred": yp, "model": model}
    print(f"  {name:28s}  Acc={acc:.4f}  F1={f1:.4f}")
    return model, yp

In [ ]:
print("Emission-Level Classification")
print("=" * 70)

# 1. Logistic Regression
evaluate_clf("Logistic Regression",
    LogisticRegression(max_iter=1000, random_state=SEED),
    X_c_tr_s, X_c_te_s, y_c_tr, y_c_te)

# 2. Decision Tree
dt, _ = evaluate_clf("Decision Tree",
    DecisionTreeClassifier(max_depth=15, random_state=SEED),
    X_c_tr, X_c_te, y_c_tr, y_c_te)
dt_imp = pd.Series(dt.feature_importances_, index=clf_features)

# 3. Random Forest
rf, _ = evaluate_clf("Random Forest",
    RandomForestClassifier(n_estimators=200, max_depth=20, random_state=SEED, n_jobs=-1),
    X_c_tr, X_c_te, y_c_tr, y_c_te)
rf_clf_imp = pd.Series(rf.feature_importances_, index=clf_features)

# 4. Gradient Boosting
gb, _ = evaluate_clf("Gradient Boosting",
    GradientBoostingClassifier(n_estimators=200, learning_rate=0.1,
                                max_depth=5, random_state=SEED),
    X_c_tr, X_c_te, y_c_tr, y_c_te)
gb_clf_imp = pd.Series(gb.feature_importances_, index=clf_features)

# 5. AdaBoost
evaluate_clf("AdaBoost",
    AdaBoostClassifier(n_estimators=100, random_state=SEED),
    X_c_tr, X_c_te, y_c_tr, y_c_te)

# 6-7. SVM
evaluate_clf("SVM (linear)",
    SVC(kernel="linear", random_state=SEED, probability=True, max_iter=5000),
    X_c_tr_s, X_c_te_s, y_c_tr, y_c_te)
evaluate_clf("SVM (rbf)",
    SVC(kernel="rbf", random_state=SEED, probability=True),
    X_c_tr_s, X_c_te_s, y_c_tr, y_c_te)

# 8. KNN
k_scores = {}
for k in [3, 5, 7, 9, 11]:
    tmp = KNeighborsClassifier(n_neighbors=k)
    tmp.fit(X_c_tr_s, y_c_tr)
    k_scores[k] = accuracy_score(y_c_te, tmp.predict(X_c_te_s))
best_k = max(k_scores, key=k_scores.get)
print(f"  KNN by k: { {k: round(v,4) for k,v in k_scores.items()} } -> best k={best_k}")
evaluate_clf(f"KNN (k={best_k})",
    KNeighborsClassifier(n_neighbors=best_k),
    X_c_tr_s, X_c_te_s, y_c_tr, y_c_te)

# 9. Naive Bayes
evaluate_clf("Naive Bayes", GaussianNB(), X_c_tr_s, X_c_te_s, y_c_tr, y_c_te)

# 10. MLP
evaluate_clf("MLP Neural Network",
    MLPClassifier(hidden_layer_sizes=(128, 64, 32), max_iter=500,
                  random_state=SEED, early_stopping=True),
    X_c_tr_s, X_c_te_s, y_c_tr, y_c_te)

print(f"\nAll {len(results)} classifiers trained!")

## 6. CO₂ Growth-Rate Forecasting

Predict `co2_growth_prct` (year-over-year CO₂ change %) using lagged features.

In [ ]:
# Build lagged features per country
gr_df = df[df["co2_growth_prct"].notna() & (df["gdp"] > 0)].copy()
gr_df = gr_df.sort_values(["country","year"])

# Lag-1 CO2 and GDP
gr_df["co2_lag1"]  = gr_df.groupby("country")["co2"].shift(1)
gr_df["gdp_lag1"]  = gr_df.groupby("country")["gdp"].shift(1)
gr_df["pop_lag1"]  = gr_df.groupby("country")["population"].shift(1)
gr_df["co2_diff"]  = gr_df["co2"] - gr_df["co2_lag1"]
gr_df = gr_df.dropna(subset=["co2_lag1"])
print(f"Growth-rate dataset: {len(gr_df):,} rows")

gr_features = ["co2_lag1","gdp_lag1","pop_lag1","gdp_per_capita",
               "coal_share","oil_share","gas_share","year"]
X_g = gr_df[gr_features].values
y_g = gr_df["co2_growth_prct"].values

# Clip extreme outliers for stability
y_g = np.clip(y_g, np.percentile(y_g, 1), np.percentile(y_g, 99))

X_g_tr, X_g_te, y_g_tr, y_g_te = train_test_split(
    X_g, y_g, test_size=0.2, random_state=SEED)
sc_g = StandardScaler()
X_g_tr_s = sc_g.fit_transform(X_g_tr)
X_g_te_s = sc_g.transform(X_g_te)
print(f"Train: {len(y_g_tr):,}  |  Test: {len(y_g_te):,}")

print("\nGrowth-Rate Regression:")
print("=" * 80)
evaluate_reg("GR \u2013 Linear", LinearRegression(), X_g_tr_s, X_g_te_s, y_g_tr, y_g_te, growth_results)
evaluate_reg("GR \u2013 Ridge", Ridge(alpha=10.0), X_g_tr_s, X_g_te_s, y_g_tr, y_g_te, growth_results)
evaluate_reg("GR \u2013 Random Forest",
    RandomForestRegressor(n_estimators=200, max_depth=10, random_state=SEED, n_jobs=-1),
    X_g_tr, X_g_te, y_g_tr, y_g_te, growth_results)
evaluate_reg("GR \u2013 Gradient Boosting",
    GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=5, random_state=SEED),
    X_g_tr, X_g_te, y_g_tr, y_g_te, growth_results)

fig, ax = plt.subplots(figsize=(10, 5))
gr_comp = pd.DataFrame({n: {k: v for k, v in d.items() if k not in ("y_pred","model")}
                         for n, d in growth_results.items()}).T.sort_values("r2")
gr_comp["r2"].plot(kind="barh", ax=ax, color="coral")
ax.set_title("Growth-Rate Forecasting \u2013 R\u00b2 Comparison", fontweight="bold")
ax.set_xlabel("R\u00b2")
fig.tight_layout()
save_plot(fig, "growth_regression")

best_gr = max(growth_results, key=lambda k: growth_results[k]["r2"])
print(f"\nBest: {best_gr} (R\u00b2 = {growth_results[best_gr]['r2']:.4f})")

## 7. Country Clustering (Unsupervised)

Group countries by their emission profile using K-Means, PCA, and DBSCAN.

In [ ]:
# Aggregate latest-year data per country
cl_df = latest[(latest["co2"].notna()) & (latest["population"] > 0)].copy()
cl_features = ["co2","co2_per_capita","coal_share","oil_share","gas_share",
               "gdp_per_capita","log_co2","log_population"]
cl_df = cl_df.dropna(subset=cl_features)
print(f"Clustering: {len(cl_df)} countries")

X_cl = cl_df[cl_features].values
sc_cl = StandardScaler()
X_cl_s = sc_cl.fit_transform(X_cl)

# Elbow + Silhouette
inertias, sils = [], []
K_range = range(2, 11)
for k in K_range:
    km = KMeans(n_clusters=k, random_state=SEED, n_init=10)
    lbl = km.fit_predict(X_cl_s)
    inertias.append(km.inertia_)
    sils.append(silhouette_score(X_cl_s, lbl))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].plot(list(K_range), inertias, "bo-")
axes[0].set_title("Elbow Method", fontweight="bold")
axes[0].set_xlabel("k"); axes[0].set_ylabel("Inertia")
axes[1].plot(list(K_range), sils, "ro-")
axes[1].set_title("Silhouette Score", fontweight="bold")
axes[1].set_xlabel("k"); axes[1].set_ylabel("Score")
fig.tight_layout()
save_plot(fig, "elbow_silhouette")

best_k_cl = list(K_range)[int(np.argmax(sils))]
print(f"Best k = {best_k_cl} (silhouette = {max(sils):.4f})")
km_final = KMeans(n_clusters=best_k_cl, random_state=SEED, n_init=10)
cl_df["Cluster"] = km_final.fit_predict(X_cl_s)

# PCA
pca = PCA(n_components=2, random_state=SEED)
X_pca = pca.fit_transform(X_cl_s)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
sc1 = axes[0].scatter(X_pca[:,0], X_pca[:,1], c=cl_df["Cluster"].values,
                       cmap="Set2", s=40, alpha=0.8)
# label top emitters
for _, row in cl_df.nlargest(8, "co2").iterrows():
    idx = cl_df.index.get_loc(row.name)
    axes[0].annotate(row["country"], (X_pca[idx, 0], X_pca[idx, 1]),
                     fontsize=7, alpha=0.8)
axes[0].set_title(f"K-Means (k={best_k_cl}) \u2013 PCA", fontweight="bold")
axes[0].set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
axes[0].set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
plt.colorbar(sc1, ax=axes[0], label="Cluster")

# DBSCAN
dbscan = DBSCAN(eps=2.0, min_samples=3)
db_lbl = dbscan.fit_predict(X_cl_s)
n_db = len(set(db_lbl)) - (1 if -1 in db_lbl else 0)
noise = (db_lbl == -1).sum()
sc2 = axes[1].scatter(X_pca[:,0], X_pca[:,1], c=db_lbl, cmap="Set1", s=40, alpha=0.8)
axes[1].set_title(f"DBSCAN ({n_db} clusters, {noise} noise)", fontweight="bold")
axes[1].set_xlabel("PC1"); axes[1].set_ylabel("PC2")
plt.colorbar(sc2, ax=axes[1], label="Cluster")
fig.tight_layout()
save_plot(fig, "clustering_results")

print("\nCluster profiles:")
profile = cl_df.groupby("Cluster")[["co2","co2_per_capita","coal_share","gdp_per_capita"]].agg(["mean","count"])
print(profile.round(3).to_string())

## 8. Hyperparameter Tuning

GridSearchCV (Random Forest) and RandomizedSearchCV (Gradient Boosting)
for emission-level classification.

In [ ]:
print("GridSearchCV: Random Forest ...")
grid_rf = GridSearchCV(
    RandomForestClassifier(random_state=SEED, n_jobs=-1),
    {"n_estimators": [100, 200], "max_depth": [10, 15, 20], "min_samples_split": [2, 5]},
    cv=3, scoring="f1_weighted", n_jobs=-1, verbose=0)
grid_rf.fit(X_c_tr, y_c_tr)
print(f"  Best params: {grid_rf.best_params_}")
print(f"  Best CV F1:  {grid_rf.best_score_:.4f}")

yp_rf = grid_rf.predict(X_c_te)
results["RF (Tuned)"] = {
    "accuracy": accuracy_score(y_c_te, yp_rf),
    "precision": precision_score(y_c_te, yp_rf, average="weighted", zero_division=0),
    "recall": recall_score(y_c_te, yp_rf, average="weighted", zero_division=0),
    "f1_score": f1_score(y_c_te, yp_rf, average="weighted", zero_division=0),
    "y_pred": yp_rf, "model": grid_rf.best_estimator_}
print(f"  Test Acc:    {results['RF (Tuned)']['accuracy']:.4f}")

print("\nRandomizedSearchCV: Gradient Boosting ...")
rand_gb = RandomizedSearchCV(
    GradientBoostingClassifier(random_state=SEED),
    {"n_estimators": [100, 200, 300], "learning_rate": [0.01, 0.05, 0.1, 0.2],
     "max_depth": [3, 5, 7, 10], "min_samples_split": [2, 5, 10]},
    n_iter=20, cv=3, scoring="f1_weighted", random_state=SEED, n_jobs=-1, verbose=0)
rand_gb.fit(X_c_tr, y_c_tr)
print(f"  Best params: {rand_gb.best_params_}")

yp_gb = rand_gb.predict(X_c_te)
results["GB (Tuned)"] = {
    "accuracy": accuracy_score(y_c_te, yp_gb),
    "precision": precision_score(y_c_te, yp_gb, average="weighted", zero_division=0),
    "recall": recall_score(y_c_te, yp_gb, average="weighted", zero_division=0),
    "f1_score": f1_score(y_c_te, yp_gb, average="weighted", zero_division=0),
    "y_pred": yp_gb, "model": rand_gb.best_estimator_}
print(f"  Test Acc:    {results['GB (Tuned)']['accuracy']:.4f}")
print("\nHyperparameter tuning complete!")

## 9. Cross-Validation, Confusion Matrices & Learning Curves

In [ ]:
# ---- 5-Fold Cross-Validation ----
print("5-Fold Cross-Validation (Emission Classification)")
print("=" * 70)
cv_models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=SEED),
    "Decision Tree": DecisionTreeClassifier(max_depth=15, random_state=SEED),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=SEED, n_jobs=-1),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=SEED),
    "Naive Bayes": GaussianNB(),
}
cv_data = []
for name, model in cv_models.items():
    acc_cv = cross_val_score(model, X_c_tr_s, y_c_tr, cv=5, scoring="accuracy")
    f1_cv  = cross_val_score(model, X_c_tr_s, y_c_tr, cv=5, scoring="f1_weighted")
    cv_data.append({"Model": name, "Acc_mean": acc_cv.mean(), "Acc_std": acc_cv.std(),
                    "F1_mean": f1_cv.mean(), "F1_std": f1_cv.std()})
    print(f"  {name:25s}  Acc={acc_cv.mean():.4f}\u00b1{acc_cv.std():.4f}  "
          f"F1={f1_cv.mean():.4f}\u00b1{f1_cv.std():.4f}")
cv_df = pd.DataFrame(cv_data)

fig, ax = plt.subplots(figsize=(12, 5))
x = np.arange(len(cv_df)); w = 0.35
ax.bar(x-w/2, cv_df["Acc_mean"], w, yerr=cv_df["Acc_std"], capsize=3,
       label="Accuracy", color="steelblue")
ax.bar(x+w/2, cv_df["F1_mean"], w, yerr=cv_df["F1_std"], capsize=3,
       label="F1", color="coral")
ax.set_xticks(x); ax.set_xticklabels(cv_df["Model"], rotation=30, ha="right")
ax.set_title("Cross-Validation: Model Comparison", fontweight="bold")
ax.set_ylabel("Score"); ax.legend(); ax.set_ylim(0, 1.05)
fig.tight_layout()
save_plot(fig, "cv_comparison")

# ---- Feature Importance ----
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
for i, (name, imp) in enumerate([("Decision Tree", dt_imp),
                                  ("Random Forest", rf_clf_imp),
                                  ("Gradient Boosting", gb_clf_imp)]):
    imp.sort_values().plot(kind="barh", ax=axes[i], color="steelblue")
    axes[i].set_title(f"{name}\nFeature Importance", fontweight="bold")
    axes[i].set_xlabel("Importance")
fig.tight_layout()
save_plot(fig, "feature_importance")

# ---- Confusion Matrices ----
cm_models = {k: v for k, v in results.items() if "y_pred" in v and "Tuned" not in k}
n = len(cm_models); ncols = 3; nrows = (n + ncols - 1) // ncols
fig, axes = plt.subplots(nrows, ncols, figsize=(6*ncols, 5*nrows))
axes_f = axes.flatten()
for i, (name, data) in enumerate(cm_models.items()):
    cm = confusion_matrix(y_c_te, data["y_pred"])
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=axes_f[i],
                xticklabels=class_names, yticklabels=class_names)
    axes_f[i].set_title(f"{name}\nAcc={data['accuracy']:.3f}",
                        fontsize=10, fontweight="bold")
    axes_f[i].set_xlabel("Predicted"); axes_f[i].set_ylabel("Actual")
for i in range(n, len(axes_f)):
    axes_f[i].set_visible(False)
fig.tight_layout()
save_plot(fig, "confusion_matrices")

# ---- Learning Curves ----
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
lc_models = [
    ("Random Forest", RandomForestClassifier(n_estimators=100, random_state=SEED, n_jobs=-1)),
    ("Gradient Boosting", GradientBoostingClassifier(n_estimators=100, random_state=SEED)),
    ("Logistic Regression", LogisticRegression(max_iter=1000, random_state=SEED)),
]
for i, (name, model) in enumerate(lc_models):
    sizes, tr_sc, va_sc = learning_curve(
        model, X_c_tr_s, y_c_tr, cv=5,
        train_sizes=np.linspace(0.1, 1.0, 8), scoring="accuracy", n_jobs=-1)
    axes[i].plot(sizes, tr_sc.mean(1), "o-", label="Train", color="steelblue")
    axes[i].fill_between(sizes, tr_sc.mean(1)-tr_sc.std(1),
                         tr_sc.mean(1)+tr_sc.std(1), alpha=0.1, color="steelblue")
    axes[i].plot(sizes, va_sc.mean(1), "o-", label="Validation", color="coral")
    axes[i].fill_between(sizes, va_sc.mean(1)-va_sc.std(1),
                         va_sc.mean(1)+va_sc.std(1), alpha=0.1, color="coral")
    axes[i].set_title(f"Learning Curve: {name}", fontweight="bold")
    axes[i].set_xlabel("Training Size"); axes[i].set_ylabel("Accuracy")
    axes[i].legend(loc="lower right"); axes[i].set_ylim(0.3, 1.05)
fig.tight_layout()
save_plot(fig, "learning_curves")

print("\nAll evaluation plots saved!")

## 10. Ensemble Methods & Final Comparison

In [ ]:
# Voting Classifier
print("Training Voting Classifier ...")
voting = VotingClassifier(estimators=[
    ("lr", LogisticRegression(max_iter=1000, random_state=SEED)),
    ("rf", RandomForestClassifier(n_estimators=100, random_state=SEED, n_jobs=-1)),
    ("gb", GradientBoostingClassifier(n_estimators=100, random_state=SEED)),
], voting="soft", n_jobs=-1)
evaluate_clf("Voting Ensemble", voting, X_c_tr_s, X_c_te_s, y_c_tr, y_c_te)

# Stacking Classifier
print("Training Stacking Classifier ...")
stacking = StackingClassifier(estimators=[
    ("rf", RandomForestClassifier(n_estimators=100, random_state=SEED, n_jobs=-1)),
    ("gb", GradientBoostingClassifier(n_estimators=100, random_state=SEED)),
    ("knn", KNeighborsClassifier(n_neighbors=best_k)),
], final_estimator=LogisticRegression(max_iter=1000, random_state=SEED),
   cv=3, n_jobs=-1)
evaluate_clf("Stacking Ensemble", stacking, X_c_tr_s, X_c_te_s, y_c_tr, y_c_te)

# ---- Final comparison table ----
rows = []
for name, data in results.items():
    rows.append({"Model": name, "Accuracy": data["accuracy"],
                 "Precision": data["precision"], "Recall": data["recall"],
                 "F1 Score": data["f1_score"]})
comp_df = pd.DataFrame(rows).sort_values("F1 Score", ascending=False)
print("\nFinal Model Comparison:")
print(comp_df.to_string(index=False))

fig, ax = plt.subplots(figsize=(14, 7))
comp_s = comp_df.sort_values("F1 Score")
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.9, len(comp_s)))
ax.barh(comp_s["Model"], comp_s["F1 Score"], color=colors)
ax.set_xlabel("F1 Score (Weighted)"); ax.set_xlim(0, 1.05)
ax.set_title("All Models \u2013 F1 Score Comparison", fontweight="bold")
for i, (_, row) in enumerate(comp_s.iterrows()):
    ax.text(row["F1 Score"]+0.005, i, f"{row['F1 Score']:.4f}", va="center", fontsize=9)
fig.tight_layout()
save_plot(fig, "model_comparison")

best_name = comp_df.iloc[0]["Model"]
best_f1 = comp_df.iloc[0]["F1 Score"]
print(f"\nBest model: {best_name} (F1 = {best_f1:.4f})")

## 11. HTML Report Generation

In [ ]:
css = """
<style>
* { margin:0; padding:0; box-sizing:border-box; }
body { font-family:'Segoe UI',system-ui,sans-serif; background:#f0f2f5; color:#333; line-height:1.6; }
.container { max-width:1200px; margin:0 auto; padding:20px; }
header { background:linear-gradient(135deg,#1b4332,#2d6a4f); color:#fff; padding:40px; border-radius:12px; margin-bottom:30px; text-align:center; }
header h1 { font-size:2.2em; margin-bottom:10px; }
header p  { font-size:1.1em; opacity:.9; }
.card { background:#fff; border-radius:12px; padding:30px; margin-bottom:25px; box-shadow:0 2px 10px rgba(0,0,0,.08); }
.card h2 { color:#1b4332; border-bottom:3px solid #2d6a4f; padding-bottom:10px; margin-bottom:20px; }
.card h3 { color:#2d6a4f; margin:15px 0 10px; }
table { width:100%; border-collapse:collapse; margin:15px 0; }
th,td { padding:10px 14px; text-align:left; border-bottom:1px solid #eee; }
th { background:#f8f9fa; font-weight:600; color:#1b4332; }
tr:hover { background:#f8f9fa; }
.dashboard { display:grid; grid-template-columns:repeat(auto-fit,minmax(180px,1fr)); gap:15px; margin-bottom:25px; }
.stat-box { background:#fff; border-radius:10px; padding:20px; text-align:center; box-shadow:0 2px 8px rgba(0,0,0,.08); }
.stat-box .number { font-size:2em; font-weight:700; color:#2d6a4f; }
.stat-box .label  { font-size:.9em; color:#666; margin-top:5px; }
.plot-img { max-width:100%; border-radius:8px; margin:15px 0; box-shadow:0 2px 8px rgba(0,0,0,.1); }
.explanation { background:#d8f3dc; border-left:4px solid #2d6a4f; padding:15px; border-radius:0 8px 8px 0; margin:15px 0; }
.best { color:#27ae60; font-weight:700; }
footer { text-align:center; padding:30px; color:#999; font-size:.9em; }
</style>
"""

html = []
html.append("<!DOCTYPE html><html lang='en'>")
html.append("<head><meta charset='UTF-8'>")
html.append("<meta name='viewport' content='width=device-width,initial-scale=1.0'>")
html.append("<title>Global CO\u2082 Emissions \u2013 ML Report</title>")
html.append(css)
html.append("</head><body><div class='container'>")

# ---- Header ----
html.append("<header>")
html.append("<h1>\U0001f30d Global CO\u2082 Emissions \u2013 ML Analysis Report</h1>")
html.append("<p>Comprehensive Machine Learning Analysis of Global Carbon Emissions Data</p>")
html.append(f"<p>{len(raw):,} records &middot; {raw['country'].nunique()} countries &middot; "
            f"{raw['year'].min()}\u2013{raw['year'].max()}</p>")
html.append("</header>")

# ---- Dashboard ----
best_clf_name = comp_df.iloc[0]["Model"]
best_clf_f1   = comp_df.iloc[0]["F1 Score"]
best_reg_name = max(regression_results, key=lambda k: regression_results[k]["r2"])
best_reg_r2   = regression_results[best_reg_name]["r2"]
best_gr_name  = max(growth_results, key=lambda k: growth_results[k]["r2"])
best_gr_r2    = growth_results[best_gr_name]["r2"]

html.append("<div class='dashboard'>")
for label, value in [
    ("Country-Years", f"{len(raw):,}"),
    ("Countries", str(raw["country"].nunique())),
    ("ML Models", str(len(results) + len(regression_results) + len(growth_results))),
    ("Best Clf F1", f"{best_clf_f1:.4f}"),
    ("Best Reg R\u00b2", f"{best_reg_r2:.4f}"),
    ("Plots Generated", str(len(report_images))),
]:
    html.append(f"<div class='stat-box'><div class='number'>{value}</div>"
                f"<div class='label'>{label}</div></div>")
html.append("</div>")

# ---- Section 1: Dataset ----
html.append("<div class='card'><h2>\U0001f4cb 1. Dataset Overview</h2>")
html.append("<p>This analysis uses the <strong>Our World in Data</strong> global CO\u2082 dataset "
            "covering emissions by country from <strong>1750 to 2021</strong>. The dataset includes "
            "79 features covering emissions by fuel source, GDP, population, temperature change, "
            "and land-use change.</p>")
html.append("<div class='explanation'><strong>Preprocessing:</strong> "
            "Filtered to 1950+, removed aggregate regions (e.g. \u2018Africa\u2019, \u2018World\u2019), "
            "engineered fuel-mix shares, GDP per capita, emission levels, and log-transforms.</div>")
html.append("</div>")

# ---- Section 2: EDA ----
html.append("<div class='card'><h2>\U0001f4ca 2. Exploratory Data Analysis</h2>")
for name, caption in [
    ("global_co2_timeline", "Global CO\u2082 emissions trajectory (1950\u20132021)"),
    ("top10_emitters", "Top 10 CO\u2082-emitting countries in the latest year"),
    ("co2_by_source", "Global CO\u2082 breakdown by fuel source over time"),
    ("co2_per_capita_dist", "Per-capita CO\u2082 distribution and emission-level breakdown"),
    ("correlation_heatmap", "Pearson correlation between numeric features"),
    ("top5_timeline", "Emission timelines for the 5 largest emitters"),
]:
    if name in report_images:
        html.append(f"<h3>{caption}</h3>")
        html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images[name]}'>")
html.append("</div>")

# ---- Section 3: Regression ----
html.append("<div class='card'><h2>\U0001f4c8 3. CO\u2082 Emissions Prediction (Regression)</h2>")
html.append("<p>Predicting total CO\u2082 emissions from economic, demographic, and energy features.</p>")
html.append("<table><tr><th>Model</th><th>R\u00b2</th><th>MAE</th><th>RMSE</th></tr>")
for name in sorted(regression_results, key=lambda k: -regression_results[k]["r2"]):
    d = regression_results[name]
    cls = " class='best'" if name == best_reg_name else ""
    html.append(f"<tr{cls}><td>{name}</td><td>{d['r2']:.4f}</td>"
                f"<td>{d['mae']:.2f}</td><td>{d['rmse']:.2f}</td></tr>")
html.append("</table>")
if "regression_results" in report_images:
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['regression_results']}'>")
if "regression_feature_importance" in report_images:
    html.append("<h3>Feature Importance</h3>")
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['regression_feature_importance']}'>")
html.append("</div>")

# ---- Section 4: Classification ----
html.append("<div class='card'><h2>\U0001f916 4. Emission-Level Classification</h2>")
html.append("<p>Classifying country-year records into Low / Medium / High / Very High "
            "per-capita CO\u2082 emission levels.</p>")
html.append("<table><tr><th>Model</th><th>Accuracy</th><th>Precision</th>"
            "<th>Recall</th><th>F1 Score</th></tr>")
for _, row in comp_df.iterrows():
    cls = " class='best'" if row["Model"] == best_clf_name else ""
    html.append(f"<tr{cls}><td>{row['Model']}</td><td>{row['Accuracy']:.4f}</td>"
                f"<td>{row['Precision']:.4f}</td><td>{row['Recall']:.4f}</td>"
                f"<td>{row['F1 Score']:.4f}</td></tr>")
html.append("</table>")
if "confusion_matrices" in report_images:
    html.append("<h3>Confusion Matrices</h3>")
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['confusion_matrices']}'>")
if "model_comparison" in report_images:
    html.append("<h3>Model Comparison</h3>")
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['model_comparison']}'>")
html.append("</div>")

# ---- Section 5: Growth Rate ----
html.append("<div class='card'><h2>\U0001f4c9 5. CO\u2082 Growth-Rate Forecasting</h2>")
html.append("<p>Predicting year-over-year CO\u2082 change (%) using lagged features.</p>")
html.append("<table><tr><th>Model</th><th>R\u00b2</th><th>MAE</th><th>RMSE</th></tr>")
for name in sorted(growth_results, key=lambda k: -growth_results[k]["r2"]):
    d = growth_results[name]
    cls = " class='best'" if name == best_gr_name else ""
    html.append(f"<tr{cls}><td>{name}</td><td>{d['r2']:.4f}</td>"
                f"<td>{d['mae']:.2f}</td><td>{d['rmse']:.2f}</td></tr>")
html.append("</table>")
if "growth_regression" in report_images:
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['growth_regression']}'>")
html.append("</div>")

# ---- Section 6: Clustering ----
html.append("<div class='card'><h2>\U0001f9e9 6. Country Clustering</h2>")
html.append(f"<p>K-Means identified <strong>{best_k_cl} country groups</strong> "
            f"based on emission profiles (silhouette = {max(sils):.4f}).</p>")
if "elbow_silhouette" in report_images:
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['elbow_silhouette']}'>")
if "clustering_results" in report_images:
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['clustering_results']}'>")
html.append("</div>")

# ---- Section 7: CV & Learning ----
html.append("<div class='card'><h2>\u2696\ufe0f 7. Cross-Validation & Learning Curves</h2>")
if "cv_comparison" in report_images:
    html.append("<h3>5-Fold Cross-Validation</h3>")
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['cv_comparison']}'>")
if "feature_importance" in report_images:
    html.append("<h3>Feature Importance (Classification)</h3>")
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['feature_importance']}'>")
if "learning_curves" in report_images:
    html.append("<h3>Learning Curves</h3>")
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['learning_curves']}'>")
html.append("</div>")

# ---- Section 8: Conclusions ----
html.append("<div class='card'><h2>\U0001f3af 8. Key Findings & Conclusions</h2>")
html.append("<div class='explanation'>")
html.append("<h3>CO\u2082 Regression</h3>")
html.append(f"<p><strong>{best_reg_name}</strong> achieved R\u00b2 = {best_reg_r2:.4f}. "
            "Source-specific CO\u2082 columns (coal, oil, gas) are the strongest predictors, "
            "as total CO\u2082 is largely their sum. Population and GDP provide additional signal.</p>")
html.append("<h3>Emission Classification</h3>")
html.append(f"<p><strong>{best_clf_name}</strong> achieved F1 = {best_clf_f1:.4f}. "
            "Tree-based models excel at distinguishing emission levels. "
            "Total CO\u2082, per-capita metrics, and fuel-mix shares are the key discriminators.</p>")
html.append("<h3>Growth-Rate Forecasting</h3>")
html.append(f"<p><strong>{best_gr_name}</strong> achieved R\u00b2 = {best_gr_r2:.4f}. "
            "Year-over-year CO\u2082 change is inherently noisy and harder to predict; "
            "lagged CO\u2082 levels and economic indicators provide moderate predictive power.</p>")
html.append("<h3>Country Clustering</h3>")
html.append(f"<p>K-Means identified {best_k_cl} distinct country clusters. "
            "Countries group primarily by total emission volume and per-capita intensity, "
            "revealing a clear divide between industrialised high-emitters and developing nations.</p>")
html.append("</div></div>")

# ---- Footer ----
html.append("<footer>")
html.append("<p>Generated with scikit-learn, pandas, matplotlib, seaborn</p>")
html.append("<p>Data: Our World in Data \u2013 Global CO\u2082 and Greenhouse Gas Emissions</p>")
html.append("</footer></div></body></html>")

# ---- Write ----
report_path = "outputs/co2_ml_report.html"
with open(report_path, "w", encoding="utf-8") as f:
    f.write("\n".join(html))

size_kb = os.path.getsize(report_path) / 1024
print(f"\u2705 HTML Report generated: {report_path}")
print(f"   File size: {size_kb:.1f} KB")
print(f"   Embedded images: {len(report_images)}")
print(f"\n\U0001f389 Analysis complete! Open the HTML file to view the full report.")